# Recurrent Neural Networks (RNNs) for Classification and Language Modelling  

In this notebook we are going to build state-of-the art RNN models for text classification (sentiment analysis) and Language Modelling. We will look into the details of building these models with RNNs and how the performance of those models could be measured efficiently. 

**NOTE:** Training on the whole corpora in this lab is time consuming on CPU. Make sure that you switch to a GPU runtime in Colab





## RNNs Recap

RNNs are designed to make use of sequential data, when the current step has some kind of relation with the previous steps. This makes them ideal for applications with a time component (audio, time-series data) and natural language. RNNs are networks for which value of a unit depends on its own previous output as input.

An input vector representing the current input element $x_t$ is multiplied by a weight matrix $W$ and then passed through an activation function to compute an activation value for a layer of hidden units. This value is, in turn, used to calculate the output, $y_t$. 

$h_t = g(Uh_{t-1}+Wx_t)$

$y_t = a(Vh_t)$

The hidden layer from the previous time step $h_{t-1}$ provides a form of memory, or context, that encodes earlier processing and informs the decisions to be made at later points in time. $U$ determines how the network should make use of past context. RNNs do not impose any limit on this prior context. The context includes information dating back to the beginning of the sequence. Three sets of weights are updated at each timestep: $W$, $U$ and $V$.



## RNNs for Text Classification Task

We will again build a model for sentiment analysis. But now we aim to classify the input text into 3 classes: positive, negative or neutral class. For example, a positive sentence "I loved the movie", a negative "I hated the movie" and a neutral "The movie was about Australia". Usually this is the last RNN hidden state is the one that summarises the whole language sequence.

**Q: Why RNNs better than FFNNs?**

In [ ]:
import torch
from torchtext import data
# This time we will work with a dataset from the torchtext package consists of data processing utilities and popular datasets for NLP
from torchtext import datasets
import random
import torch.nn as nn
import time
import math
import nltk
nltk.download('punkt')

# We fix the seeds to get consistent results for each training.

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Helper function to print time between epochs
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We will experiment with a widely used Stanford Treebank dataset and will predict sentiment of movie reviews. Our data will be classified in three labels: positive, negative and neutral. We take the standard split.

In [ ]:
# define a tokenizer
tokenize = lambda s : nltk.word_tokenize(s)

# With TorchText Field we define how our data will be processed

TEXT = data.Field(tokenize=tokenize, lower=True)
LABEL = data.LabelField(dtype = torch.long)

train_data, valid_data, test_data = datasets.SST.splits(
            TEXT, LABEL)

# Print stat over the data

print('train.fields:', train_data.fields)
print('len(train):', len(train_data))
print('vars(train[0]):', vars(train_data[0]))

Print out the first two tokens in the vocabulary. 

**Q: Why do we map unknown tokens to a special UNK token? Do you think the network will learn a useful embedding for that? If not, how can you let the network to learn an embedding for it?**


In [ ]:

# here we can use pre-trained embeddings, though too heavy for this environment
#TEXT.build_vocab(train_data, vectors="glove.6B.50d")
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

print('Text Vocabulary Length', len(TEXT.vocab))
print ("Label Vocabulary Length: ", len(LABEL.vocab))

#We can display the most common words in the vocabulary and their frequencies

print(TEXT.vocab.freqs.most_common(20))

#We can also see the vocabulary directly using the stoi (string to int)

print(LABEL.vocab.stoi)

In [ ]:
BATCH_SIZE = 64

# place the tensors on the GPU if available

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# BucketIterator is an iterator that will return a batch of examples of similar lengths, minimizing the amount of padding per example.
# Padding refers to fixing the length of inputs (adding a reserved token a certain amount of times to match certain length), usually to the max length within a batch. For example:
# i         like  this  movie <pad>
# the       movie is    very  good
# excellent !     <pad> <pad> <pad>

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
            (train_data, valid_data, test_data), 
            batch_size = BATCH_SIZE,
            sort_within_batch = True,
            device = device)

In [ ]:
print(train_iterator)

for batch in train_iterator:
    demo_batch = batch
    break
    
print(demo_batch)

print()

# Note that demo_batch.text has a shape of [sentence length x batch size]
print("Demo batch `text` shape:", demo_batch.text.shape)
# We can simply reshape this into the more familiar [batch size x sentence length]
print("Demo batch `text` transpose shape:", demo_batch.text.T.shape)
print("Demo batch `text` sample: \n", demo_batch.text.T[:3, :])

print()

print("Demo batch `label` shape:", demo_batch.label.shape)
# shape(demo_batch.label.shape) = [batch size]
print("Demo batch `label` sample:", demo_batch.label[:3])

## Building the RNN

Our RNN class is a sub-class of `nn.Module`. Within the `__init__` method, we define the layers of the module:

- Our first layer is an embedding layer (look-up layer). This layer could be initialized with pre-trained embeddings or trained together with other layers.
 
- The next layer is an RNN layer.

- Finally, the last linear layer is the output layer for the classification task. This layer receives the last hidden state from the RNN and outputs logits of the output dimensionality.

**Q. Fill in the gaps in the code below.**

In [ ]:
class RNN(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 bidirectional, dropout, pad_idx):

        super().__init__()

        self.bidirectional = bidirectional
        self.hidden_dim = hidden_dim
        
        # Here, we initialize our model with pre-trained embeddings (50D pre-trained GloVe embeddings in our case).
        # This layer will fine-tune these embeddings, specific to this model/dataset.
        #self.embedding = nn.Embedding.from_pretrained(TEXT.vocab.vectors)

        # We can also train the embeddings from scratch:
        self.embedding =  

        
        # An RNN layer. We specify that the batch dimension goes first
        # We have a bidirectional flag which indicates whether the model is unidirectional or bidirectional
        # RNNs can be stacked - i.e. have multiple layers. Here, we will only look at the 1 layer case.
        self.rnn = nn.RNN(embedding_dim,
                          hidden_dim,
                          batch_first=True,
                          bidirectional=bidirectional,
                          num_layers=1)

          # The linear layer takes the final hidden state and feeds it through a fully connected layer.
          # The dimensionality of the output is equal to the output class count.
          # For classification in a bidirectional RNN we concatenate:
            #  - The last hidden state from the forward RNN (obtained from final word of the sentence)
            #  - The last hidden state from the backward RNN (obtained from the first word of the sentence)
          # Due to the concatenation, our hidden size is doubled.
        
        if self.bidirectional:
            linear_hidden_in = hidden_dim * 2
        else:
            linear_hidden_in = hidden_dim

        # The classification (linear) layer
        self.fc = nn.Linear(linear_hidden_in, output_dim)
        

        # We apply dropout technique that sets a random set of activations of a layer to zero.
        # This prevents the network from learning to rely on specific weights and helps to prevent overfitting. 
        # Note that the dropout layer is only used during training, and not during test time.
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text):

        # ACRONYMS:
          # B = Batch size
          # T = Max sentence length
          # E = Embedding dimension
          # D = Hidden dimension
          # O = Output dimension

        # shape(text) = [B, T]

        embedded = self.dropout(self.embedding(text))
        # shape(embedded) = [B, T, E]
        
        # An RNN in PyTorch returns two values:
        # (1) All hidden states of the last RNN layer
        # (2) Hidden state of the last timestep for every layer
        # Note: we are only using 1 layer
        all_hidden, last_hidden = self.rnn(embedded)
        # shape(all_hidden) = [B, T, D*num_directions]
        # shape(last_hidden) = [num_layers*num_directions, B, D].  num_layers = 1
        # NOTE. If we were to NOT use the `batch_first` flag, shape of all_hidden would be [T, B, D*num_directions]
        
        if self.bidirectional:
            # Concat the final forward (hidden[0,:,:]) and backward (hidden[1,:,:]) hidden layers
            last_hidden = torch.cat((last_hidden[0, :, :], last_hidden[1, :, :]), dim=-1)
            # shape(last_hidden) = [B, D*2]

        else:
            last_hidden = last_hidden.squeeze(0)
            # shape(last_hidden) = [B, D]

        # Our predictions.
        logits = self.fc(self.dropout(last_hidden))
        # shape(logits) = [B, O]
        
        return logits

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 128
OUTPUT_DIM = len(LABEL.vocab)
BIDIRECTIONAL = False
DROPOUT = 0.3
# get our pad token index from the vocabulary
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]


Let's initialise the RNN now:

In [ ]:
model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX).to(device)

**Q. Fill in the gaps in the code below.**

In [ ]:
def accuracy(preds, y):
    """
    returns accuracy per batch
    """

    class_preds =  
    correct = (class_preds == y).float() # convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

**Q. Fill in the gaps in the code below.**

In [ ]:
import torch.optim as optim
def train(model, train_iterator, valid_iterator, optimizer, criterion, N_EPOCHS=10):
    optimizer = optimizer
    criterion = criterion
    model = model.to(device)
    criterion = criterion.to(device)

    for epoch in range(N_EPOCHS):
    
        start_time = time.time()

        # To ensure the dropout is "turned on" while training
        # (good practice to include in your projects even if it is not used)
        model.train()
        
        epoch_loss = 0
        epoch_acc = 0
    
        # `batch` is a tuple of Tensors: (TEXT, LABEL)
        for batch in train_iterator:
                        
            # Zero the gradients
            optimizer.zero_grad()

            text = batch.text.to(device)
            labels = batch.label.to(device)
            # shape(text) = [T, B]
            # shape(label) = [B]
            
            # We reshape text to [B, T]. 
            # This is purely so we can think about the shapes of the Tensors more consistently
            text = text.T
            
            predictions = model(text)
            
            # compute the loss
            loss =  
            print(loss)
        
            # compute training accuracy
            acc = accuracy(predictions, labels)
              
            # calculate the gradient of each parameter
            loss.backward()
        
            # update the parameters using the gradients and optimizer algorithm 
            optimizer.step()
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
        average_epoch_loss = epoch_loss / len(train_iterator)
        average_epoch_acc = epoch_acc / len(train_iterator)
        
        end_time = time.time()
        
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
        average_epoch_valid_loss, average_epoch_valid_acc = evaluate(model, valid_iterator, criterion)

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {average_epoch_loss:.3f} | Train Acc: {average_epoch_acc*100:.2f}%')
        print(f'\t Val. Loss: {average_epoch_valid_loss:.3f} |  Val. Acc: {average_epoch_valid_acc*100:.2f}%')

In [ ]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    # Turn on evaluate mode. This de-activates dropout. 
    model.eval()

    # We do not compute gradients within this block, i.e. no training
    with torch.no_grad():

        for batch in iterator:

            text = batch.text.to(device)
            labels = batch.label.to(device)

            text = text.T
            
            predictions = model(text)
            loss = criterion(predictions, labels)
            acc = accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
optimizer = optim.Adam(model.parameters())
# we use the cross-entropy loss
# note that by default losses are averaged over the minibatch
criterion = nn.CrossEntropyLoss()
train(model, train_iterator, valid_iterator, optimizer, criterion)

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## BiDirectional RNNs

In problems where all timesteps of the input sequence are available, bidirectional RNNs train two instead of one RNNs on the input sequence. The first on the input sequence as-is and the second on a reversed copy of the input sequence. Outputs for the same sequence step are then usually concatenated. This can provide additional useful context to the model.

**Q: Why is a bi-directional RNN is better than single-direction ?**

In [ ]:
BIDIRECTIONAL = True

bidirectional_model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [ ]:
optimizer = optim.Adam(bidirectional_model.parameters())
criterion = nn.CrossEntropyLoss()
train(bidirectional_model, train_iterator, valid_iterator, optimizer, criterion)

In [ ]:
test_loss, test_acc = evaluate(bidirectional_model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# Language Modelling

## N-gram language models

A language model (LM) assigns a probability to each possible next word, given a history of previous words *i.e.* the context:

  $P(w_n|w_1, w_2, \dots, w_{n-1})=P(w_n|w_1^{n-1})$

Since calculating the probability of the whole sentence is not feasible, the **Markov assumption** is introduced: each word depends only on the previous $C$ words, where $C=n-1$ for $n$-gram language models. With this assumption, the above probability is approximated as follows:

  - `1-gram ` $\approx P(w_n)$
  - `2-gram ` $\approx P(w_n|w_{n-1})$
  - `3-gram ` $\approx P(w_n|w_{n-2}, w_{n-1})$
  - `N-gram ` $\approx P(w_n|w_{n-N+1}, \dots, w_{n-2}, w_{n-1})$

The n-gram probabilities are then calculated by normalising the count of that n-gram by the count of the context/prefix:

  $P(w_n|w_{n-N+1}, \dots, w_{n-2}, w_{n-1})=\large\frac{C(w_{n-N+1}, \dots, w_{n-1}, w_{n})}{C(w_{n-N+1}, \dots, w_{n-1})}$


Smoothing is used to deal with the sparsity problem in the N-Gram LM. With smoothing the probability mass  shifted towards the less frequent words.

For example, an add-1 smoothing for a bigram model:

$P_{add-1}(w_n|w_{n-1})=\frac{C(w_{n-1},w_n)+1}{C(w_{n-1})+V}$

**Q: What is the disadvantage of smoothing?**

In [ ]:
from collections import Counter, defaultdict
import math
import copy
import random
import operator

flatten = lambda l: [item for sublist in l for item in sublist]

class NGramLM():
    def __init__(self, N):
        self.N = N
        self.vocab = set()
        self.data = []
        self.prob = {}
        self.count = defaultdict(Counter)
    
    # For N = 1, the probability is stored in a dict   P = prob[next_word]
    # For N > 1, the probability is in a nested dict   P = prob[context][next_word]
    def train(self, vocab, data, smoothing_k=0):
        self.vocab = vocab
        self.data = data
        self.smoothing_k = smoothing_k

        if self.N == 1:
            self.counts = Counter(flatten(data))
            self.prob = self.get_prob(self.counts)
        else:
            self.vocab.add('<s>')
            counts = self.count_ngram()
            
            self.prob = {}
            for context, counter in counts.items():
                self.prob[context] = self.get_prob(counter)
    
    def count_ngram(self):
        counts = defaultdict(Counter)
        for sentence in self.data:
            sentence = (self.N - 1) * ['<s>'] + sentence 
            for i in range(len(sentence)-self.N+1):
                context = sentence[i:i+self.N-1]
                context = " ".join(context)
                word = sentence[i+self.N-1]
                counts[context][word] += 1

        self.counts = counts
        return counts
        
    # normalize counts into probability(considering smoothing)
    def get_prob(self, counter):
        total = float(sum(counter.values()))
        k = self.smoothing_k
        
        prob = {}
        for word, count in counter.items():
            prob[word] = (count + k) / (total + len(self.vocab) * k)
        return prob
        
    def get_ngram_logprob(self, word, seq_len, context=""):
        if self.N == 1 and word in self.prob.keys():
            return math.log(self.prob[word]) / seq_len
        elif self.N > 1 and not self._is_unseen_ngram(context, word):
            return math.log(self.prob[context][word]) / seq_len
        else:
            # assign a small probability to the unseen ngram
            # to avoid log of zero and to penalise unseen word or context
            return math.log(1/len(self.vocab)) / seq_len
        
    def get_ngram_prob(self, word, context=""):
        if self.N == 1 and word in self.prob.keys():
            return self.prob[word]
        elif self.N > 1 and not self._is_unseen_ngram(context, word):
            return self.prob[context][word]
        elif word in self.vocab and self.smoothing_k > 0:
            # probability assigned by smoothing
            return self.smoothing_k / (sum(self.counts[context].values()) + self.smoothing_k*len(self.vocab))
        else:
            # unseen word or context
            return 0
    
    def _is_unseen_ngram(self, context, word):
        if context not in self.prob.keys() or word not in self.prob[context].keys():
            return True
        else:
            return False
    
    # generate the most probable k words
    def generate_next(self, context, k):
        contex = (self.N-1) * '<s>' + context
        context = context.split()
        ngram_context_list = context[-self.N+1:]
        ngram_context = " ".join(ngram_context_list)
        
        if ngram_context in self.prob.keys():
            candidates = self.prob[ngram_context]
            most_probable_words = sorted(candidates.items(), key=lambda kv: kv[1], reverse=True)
            for i in range(min(k, len(most_probable_words))):
                print(" ".join(context)+" "+most_probable_words[i][0]+"\t P={}".format(most_probable_words[i][1]))
        else:
            print("Unseen context!")
            
    # generate the next n words with greedy search
    def generate_next_n(self, context, n):
        contex = (self.N-1) * '<s>' + context
        context = context.split()
        ngram_context_list = context[-self.N+1:]
        ngram_context = " ".join(ngram_context_list)
        
        for i in range(n):
            try:
                candidates = self.prob[ngram_context]
                most_likely_next = max(candidates.items(), key=operator.itemgetter(1))[0]
                context += [most_likely_next]
                ngram_context_list = ngram_context_list[1:] + [most_likely_next]
                ngram_context = " ".join(ngram_context_list)
            except:
                break
        print(" ".join(context))

Let's train a bigram Language Model on the toy dataset.

In [ ]:
corpus = ["I like ice cream",
         "I like chocolate",
         "I hate beans"]
tokens = [l.strip().lower().split() + ['</s>'] for l in corpus if l.strip()]
vocab = set(flatten(tokens))
print(tokens)
print(vocab)

def print_probability(lm):
    for context in lm.vocab:
        for word in lm.vocab:
            prob = lm.get_ngram_prob(word, context)
            print("P({}\t|{}) = {}".format(word, context, prob))
        print("--------------------------")

lm = NGramLM(2)

# here we do not use smoothing

lm.train(vocab, tokens, smoothing_k=0)

print_probability(lm)


# Neural Language Modelling

We will be using the WikiText-2 corpus, which is a popular LM dataset. The WikiText language modeling dataset is a collection of texts extracted from articles on Wikipedia.
It contains about 2 million words. 

**Q: What is the difference between word embeddings and language modelling?**


**Q: How is the output of the network is computed ?**


**Q: What is the training loss for LM tasks ?**



In [ ]:
# Download the corpus
%%bash
URL="https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2"

for split in "train" "valid" "test"; do
  if [ ! -f "${split}.txt" ]; then
    echo "Downloading ${split}.txt"
    wget -q "${URL}/${split}.txt"
    # Remove empty lines
    sed -i '/^ *$/d' "${split}.txt"
    # Remove article titles starting with = and ending with =
    sed -i '/^ *= .* = $/d' "${split}".txt
  fi
done

# Prepare smaller version for fast training neural LMs
head -n 5000 < train.txt > train_small.txt

# Print the first 10 lines with line numbers
cat -n train.txt | head -n10
echo

# Print some statistics
echo -e "\n   Line,   word,   character counts"
wc *.txt

The below Vocabulary class encapsulates the *word-to-idx* and *idx-to-word* mapping that you should now be familiar with from the previous lab sessions. Read it to understand how the vocabulary is constructed from a plain text file, within the `build_from_file()` method. Special <.> markers are also included in the vocabulary.

In [ ]:
class Vocabulary(object):
  """Data structure representing the vocabulary of a corpus."""
  def __init__(self):
    # Mapping from tokens to integers
    self._word2idx = {}

    # Reverse-mapping from integers to tokens
    self.idx2word = []

    # 0-padding token
    self.add_word('<pad>')
    # Unknown words
    self.add_word('<unk>')

    self._unk_idx = self._word2idx['<unk>']

  def word2idx(self, word):
    """Returns the integer ID of the word or <unk> if not found."""
    return self._word2idx.get(word, self._unk_idx)

  def add_word(self, word):
    """Adds the `word` into the vocabulary."""
    if word not in self._word2idx:
      self.idx2word.append(word)
      self._word2idx[word] = len(self.idx2word) - 1

  def build_from_file(self, fname):
    """Builds a vocabulary from a given corpus file."""
    with open(fname) as f:
      for line in f:
        words = line.strip().split()
        for word in words:
          self.add_word(word)

  def convert_idxs_to_words(self, idxs):
    """Converts a list of indices to words."""
    return ' '.join(self.idx2word[idx] for idx in idxs)

  def convert_words_to_idxs(self, words):
    """Converts a list of words to a list of indices."""
    return [self.word2idx(w) for w in words]

  def __len__(self):
    """Returns the size of the vocabulary."""
    return len(self.idx2word)
  
  def __repr__(self):
    return "Vocabulary with {} items".format(self.__len__())

Let's construct the vocabulary for the training set.

In [ ]:
vocab = Vocabulary()
vocab.build_from_file('train_small.txt')
print(vocab)

In [ ]:
def readable_size(n):
  """Returns a readable size string for model parameters count."""
  sizes = ['K', 'M', 'G']
  fmt = ''
  size = n
  for i, s in enumerate(sizes):
    nn = n / (1000 ** (i + 1))
    if nn >= 1:
      size = nn
      fmt = sizes[i]
    else:
      break
  return '%.2f%s' % (size, fmt)

def corpus_to_tensor(_vocab, filename):
  # Final token indices
  idxs = []
  
  with open(filename) as data:
    for line in tqdm(data, ncols=80, unit=' line', desc=f'Reading {filename} '):
      line = line.strip()
      # Skip empty lines if any
      if line:
        # Split from whitespace and add sentence markers
        idxs.extend(_vocab.convert_words_to_idxs(line.split()))
  return torch.LongTensor(idxs)

train_small = corpus_to_tensor(vocab, 'train_small.txt')

valid = corpus_to_tensor(vocab, 'valid.txt')
test = corpus_to_tensor(vocab, 'test.txt')
print('\n')

print(f'Small training size in tokens: {readable_size(len(train_small))}')
print(f'Validation size in tokens: {readable_size(len(valid))}')
print(f'Test size in tokens: {readable_size(len(test))}')

## Feed-forward Language Models (FFLM)

FFLMs are similar to $n$-gram language models in the sense that the choice of $n$ is a hyperparameter for the network architecture. A basic FFLM constructs a $C=n\mathrm{-1}$ length context window before the word to be predicted. If the word embedding size is $E$, the feature vector for the context window becomes a vector of size $E\times C$, resulting from the concatenation of individual word embeddings of context words. Hence, the choice of $C$ for FFLMs, affects the number of final learnable parameters in the network.

**Q. Fill in the gaps in the code below.**



In [ ]:
class FFLM(nn.Module):
  def __init__(self, vocab_size, emb_dim, hid_dim, context_size, dropout=0.5):
    # Call parent's __init__ first
    super(FFLM, self).__init__()
    
    # Store arguments
    self.vocab_size = vocab_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.context_size = context_size

    # Create the loss, don't sum or average, we'll take care of it
    # in the training loop for logging purposes
    self.loss = nn.CrossEntropyLoss(reduction='none')

    # Create the non-linearity
    self.nonlin = torch.nn.Tanh()

    # Dropout regularizer
    self.drop = nn.Dropout(p=dropout)

    # Compute the dimension of the context vector
    self.context_dim =  
    
    # Create the embedding layer (i.e. lookup table tokens->vectors)
    self.emb = nn.Embedding(
        num_embeddings=self.vocab_size, embedding_dim=self.emb_dim,
        padding_idx=0)
 
    # This cuts the number of parameters a bit
    self.ff_ctx = nn.Linear(self.context_dim, self.hid_dim)

    # Output layer mapping from the output of `ff_ctx` to vocabulary size
    self.out = nn.Linear(self.hid_dim, self.vocab_size)

    # Purely for informational purposes: compute # of total params
    self.n_params = 0
    for param in self.parameters():
        self.n_params += np.cumprod(param.data.size())[-1]
    self.n_params = readable_size(self.n_params)
      
  def forward(self, x, y):
    """Forward-pass of the module."""
    # Shape of x is (batch_size, context_size)

    # Get the embeddings for the token indices in `x`
    embs = self.emb(x)

    # Concatenate the embeddings to form the context vector
    ctx = embs.view(embs.shape[0], -1)

    # apply ff_ctx -> non-lin -> dropout -> output layer
    ctx = self.drop(self.nonlin(self.ff_ctx(ctx)))
    logits = self.out(ctx)
    return self.loss(logits, y)

  def get_batches(self, data_tensor, batch_size=64):
    """Returns a tensor of size (n_batches, batch_size, context_size + 1)."""
    # Split data into rows of n-grams followed by the (n+1)th true label
    x_y = data_tensor.unfold(0, self.context_size + 1, step=1)

    # Get the number of training n-grams
    n_samples = x_y.size()[0]

    # hack: discard the last uneven batch for simplicity
    n_batches = n_samples // batch_size
    n_samples = n_batches * batch_size
    # Split nicely into batches, i.e. (n_batches, batch_size, context_size + 1)
    # The final element in each row is the ID of the true label to predict
    x_y = x_y[:n_samples].view(n_batches, batch_size, -1)

    # A particular batch for context_size=2 will now look like below in
    # word format. Last element for every array is the next token to be predicted
    #
    # [[<s>, cat, sat],
    #  [cat, sat, on],
    #  [sat, on,  the],
    #  [on,  the, mat],
    #   ....
    return x_y

  def train_model(self, optim, train_tensor, valid_tensor, test_tensor, n_epochs=5,
                 batch_size=64, shuffle=False):
    """Trains the model."""
    # Get batches for the training data
    batches = self.get_batches(train_tensor, batch_size)
    
    print(f'Will do {batches.size(0)} batches for an epoch.')

    for eidx in range(1, n_epochs + 1):
      start_time = time.time()
      epoch_loss = 0
      epoch_items = 0

      # Enable training mode
      self.train()

      # Shuffle the batch order or not
      if shuffle:
        batch_order = torch.randperm(batches.size(0))
      else:
        batch_order = torch.arange(batches.size(0))

      # Start training
      for iter_count, idx in enumerate(batch_order):
        batch = batches[idx].to(device)

        # split into inputs `x` and labels `y`
        x, y = batch[:, :self.context_size], batch[:, -1]

        # Clear the gradients
        optim.zero_grad()

        # loss will be a vector of size (batch_size, ) with losses per every sample
        loss = self.forward(x, y)

        # Backprop the average loss and update parameters
        loss.mean().backward()
        optim.step()

        # sum the loss for reporting, along with the denominator
        epoch_loss += loss.detach().sum()
        epoch_items += loss.numel()

        if iter_count % 1000 == 0:
          # Print progress
          loss_per_token = epoch_loss / epoch_items
          print(f'[Epoch {eidx:<3}] loss: {loss_per_token:6.2f}')

      time_spent = time.time() - start_time

      print(f'\n[Epoch {eidx:<3}] ended with train_loss: {loss_per_token:6.2f}')
      # Evaluate on valid set
      valid_loss = self.evaluate(test_set=valid_tensor)
      print(f'[Epoch {eidx:<3}] ended with valid_loss: {valid_loss:6.2f}')
      print(f'[Epoch {eidx:<3}] completed in {time_spent:.2f} seconds\n')

    # Evaluate the final model on test set
    test_loss = self.evaluate(test_set=test_tensor)
    print(f' ---> Final test set performance: {test_loss:6.2f}')

  def evaluate(self, test_set, batch_size=32):
    """Evaluates and computes perplexity for the given test set."""
    loss = 0

    # Get the batches
    batches = self.get_batches(test_set, batch_size)

    # Eval mode
    self.eval()

    with torch.no_grad():
      for batch in batches:
        batch = batch.to(device)

        # split into inputs `x` and labels `y`
        x, y = batch[:, :self.context_size], batch[:, -1]

        # loss will be a vector of size (batch_size, ) with losses per every sample
        # sum the loss for reporting, along with the denominator
        loss += self.forward(x, y).sum()
    
    # Normalize by the number of tokens in the test set
    loss /= batches.size()[:2].numel()

    # Switch back to training mode
    self.train()

    # return the loss
    return loss

  def __repr__(self):
    """String representation for pretty-printing."""
    s = super(FFLM, self).__repr__()
    return f"{s}\n# of parameters: {self.n_params}"

In [ ]:
fflm_model = FFLM(
    len(vocab),       # vocabulary size
    emb_dim=128,      # word embedding dim
    hid_dim=128,      # hidden layer dim
    context_size=2,   # C = (N-1) if you think in n-gram LM terminology
    dropout=0.4,      # dropout probability
)

# move to device
fflm_model.to(device)

# Initial learning rate for the optimizer
FFLM_INIT_LR = 0.001

# Create the optimizer
fflm_optimizer = torch.optim.Adam(fflm_model.parameters(), lr=FFLM_INIT_LR)
print(fflm_model)

fflm_model.train_model(fflm_optimizer, train_small, valid, test, n_epochs=5, batch_size=64, shuffle=True)

## Recurrent Language Models (RNNLM)

Now we switch to more complex RNN LMs which have access to large context windows.

Here for pre-processing we will use the `torchtext` package with data processing utilities and popular datasets for natural language. We will again work with the WikiText2 dataset.

**Q. Why do we need the BOS token ?**
 


In [ ]:
tokenize = lambda s : nltk.word_tokenize(s)
# With TorchText Field we define how our data will be processed
TEXT = data.Field(tokenize = tokenize, init_token = '<bos>')

train_data, valid_data, test_data = datasets.WikiText2.splits(TEXT)

# Data stats
print('train.fields', train_data.fields)
print('len(train)', len(train_data))

# Build a vocabulary out of tokens available from the pre-trained embeddings list and the vocabulary of labels
TEXT.build_vocab(train_data)
print('Text Vocabulary Length', len(TEXT.vocab))

BATCH_SIZE = 64

# place the tensors on the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Backpropagation Through Time, or BPTT, is the application of the Backpropagation training algorithm to RNNs. BPTT works by unrolling all input timesteps. Each timestep has one input timestep, one copy of the network, and one output. Errors are then calculated and accumulated for each timestep. If input sequences are comprised of thousands of timesteps, then this will be the number of derivatives required for a single update weight update. This can cause weights to vanish or explode. Truncated Backpropagation Through Time, or TBPTT, is a modified version of the BPTT where the BPTT update is performed back for a fixed number of timesteps.

In [ ]:
tokenize = lambda s : nltk.word_tokenize(s)
# With TorchText Field we define how our data will be processed
TEXT = data.Field(tokenize = tokenize, init_token = '<bos>')

train_data, valid_data, test_data = datasets.WikiText2.splits(TEXT)

# Data stats
print('train.fields', train_data.fields)
print('len(train)', len(train_data))

# Build a vocabulary out of tokens available from the pre-trained embeddings list and the vocabulary of labels
TEXT.build_vocab(train_data)
print('Text Vocabulary Length', len(TEXT.vocab))

BATCH_SIZE = 64

# place the tensors on the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class RNN(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout, pad_idx):
            
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # UNIDIRECTIONAL RNN layer: For LM modelling we do not see/have access to the right context
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
 
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)

       
    def forward(self, text, prev_hidden):
         
        # shape(text) = [B, T]
        
         
        # shape(prev_hidden) = [1, B, D] where 1 = num_layers*num_directions
   
        embedded = self.dropout(self.embedding(text))
        # shape(embedded) = [B, T, E]
        
        all_hidden, last_hidden = self.rnn(embedded, prev_hidden)        
        # shape(all_hidden) = [B, T, D]
        # shape(last_hidden) = [num layers, B, D]
 
        # Take all hidden states to produce an output word per time step
        
        logits = self.fc(self.dropout(all_hidden))
        # shape(logits) = [B, O]
            
        return logits, last_hidden

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(TEXT.vocab)
DROPOUT = 0.5
# get our pad token index from the vocabulary
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

rnn_model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            DROPOUT, 
            PAD_IDX)

import torch.optim as optim

We need to detach the hidden state or else the model will try to backpropagate to the beginning of the dataset, requiring a lot of memory.

In [ ]:
def save_hidden(hidden):
  """Wraps hidden states in new Tensors, to declare it not to need gradients. So that the initial hidden state for this batch is constant and doesn’t depend on anything."""

  if isinstance(hidden, torch.Tensor):
    return hidden.detach()
  else:
    return tuple(save_hidden(v) for v in hidden)

## Perplexity

Language is very difficult to evaluate since there is no single gold truth: one meaning could be expressed in many valid ways. Human evaluation of a language model may involve how a hypothesis satisfies the grammatical and lexical norms of a language. Human evaluation is reliable, but costly and slow. Automatic metrics are a less costly and faster option.

For example, **perplexity** can reveal if the model prefers real (=frequently observed) sentences to ‘ungrammatical/gibberish’ (or rarely observed) ones. Remember that entropy is the average number of bits to encode the information contained in a random variable, so the exponentiation of the entropy (perplexity, $e^{H}$) should be the total amount of all possible information, or more precisely, the weighted average number of choices a random variable has. We evaluate our prediction Q by testing against samples drawn from P: $PPL = e^{CrossEntropy}$.

We usually measure perplexity on an unseen (test) corpus, generally we compare a range of models using this score. The best LM is the one that generates the lowest perplexity on the test corpus.

**Q. Fill in the gaps in the code below.**

In [ ]:
def perplexity(loss_per_token):
    return  

**Q: How are language models trained ?**

**Q. Fill in the gaps in the code below.**

In [ ]:
def train(model, train_iterator, valid_iterator, optimizer, criterion, N_EPOCHS=10, is_lstm=False, force_stop=False):
    
    optimizer = optimizer
    criterion = criterion

    model = model.to(device)
    criterion = criterion.to(device)
    
    for epoch in range(N_EPOCHS):
    
        start_time = time.time()
        
        model.train()

        epoch_loss = 0
        epoch_items = 0
        
        # The `1` is the number of layers * number of directions.
        # i.e. we have 1 layer and we are moving in 1 direction
        prev_hidden = torch.zeros(1, BATCH_SIZE, HIDDEN_DIM, device=device)
  
        
        # `batch` is a tuple of Tensors: (TEXT, TARGET)
        for i, batch in enumerate(train_iterator):
            
            if force_stop:
                print("Currently processing train batch {} of {}".format(i, len(train_iterator)))
                if i % 7 == 0 and i != 0:
                    break
            
            # Zero the gradients
            optimizer.zero_grad()

            text = batch.text
            targets = batch.target
            print()
            # shape(text) = [T, B]
            # shape(target) = [T, B]
            
            # We reshape text and target to [B, T]. 
            text = text.T.to(device)
            targets = targets.T.to(device)
            # shape(text) = [B, T]
            # shape(target) = [B, T]
                        
            # Starting each batch, we detach the hidden state from how it was previously produced.
            # Otherwise the model would backpropagate all the way to beginning of the dataset.
            prev_hidden = save_hidden(prev_hidden)
            
            ## run the model
            logits, prev_hidden = model(text, prev_hidden)
            
            # Compute the loss
            # We reshape inputs to eliminate batching
            
            print("logits", logits.view(-1, OUTPUT_DIM).shape)
            print("targets", targets.reshape(-1).shape)
            
            loss = 
            # backprop the average loss and update parameters
            loss.backward()
        
            # update the parameters using the gradients and optimizer algorithm 
            ##call the optimizer
            optimizer.step()
            
            
            epoch_loss += loss.detach()
            epoch_items += loss.numel()
        
        # We compute loss per token for an epoch
        train_loss_per_token = epoch_loss / epoch_items
        # We compute perplexity
        train_ppl = perplexity(train_loss_per_token)

        end_time = time.time()        
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
        valid_loss_per_token, valid_ppl = evaluate(model, 
                                                   valid_iterator, 
                                                   criterion,
                                                   is_lstm=is_lstm,
                                                   force_stop=force_stop)

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss_per_token:.3f} | Train Perplexity: {train_ppl:.3f}')
        print(f'\t Val. Loss: {valid_loss_per_token:.3f} |  Val. Perplexity: {valid_ppl:.3f}')
        
        if force_stop:
            break

def evaluate(model, iterator, criterion, is_lstm=False, force_stop=False):
    
    model.eval()
    
    epoch_loss = 0
    epoch_items = 0
    
    # we initialise the first hidden state with zeros
    ## Initialise the previous hidden states of the RNNs
    prev_hidden = torch.zeros(1, BATCH_SIZE, HIDDEN_DIM, device=device)
 
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            
            if force_stop and i % 3 == 0 and i != 0:
                print("Currently processing valid batch {} of {}".format(i, len(train_iterator)))
                break

            text, target = batch.text, batch.target
            text, target = text.T, target.T
            logits, prev_hidden = model(text, prev_hidden)

            # compute the loss
            loss =  

            prev_hidden = save_hidden(prev_hidden)

            epoch_loss += loss.detach()
            epoch_items += loss.numel()

        loss_per_token = epoch_loss / epoch_items
        ppl = math.exp(loss_per_token)
            
        
    return loss_per_token, ppl

optimizer = optim.Adam(rnn_model.parameters())
criterion = nn.CrossEntropyLoss()
train(rnn_model, train_iterator, valid_iterator, optimizer, criterion, force_stop=True)


# Advanced: Long short-term memory architectures LSTMs vs. RNNs

The gradient signal gets smaller and smaller as it backpropagates further. It is caused by the repeated use of the recurrent weight matrix in RNN. Gradient can be viewed as a measure of the effect of the past on the future. If the gradient becomes vanishingly small over longer distances we can not capture the dependency to the past correctly. For example: "A patient with a rare sarcoma of soft tissue on the left thigh was presented to the hospital yesterday." "was presented" depends on "a patient", but they are separated by 11 words!

LSTM has two "hidden states": $c_t$  and $h_t$ . You can think of $c_t$  is the "internal" hidden state that retains important information for longer timesteps, whereas $h_t$ is the "external" hidden state that exposes that information to the outside world.

The LSTM does have the ability to remove or add information to the cell state. Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation.  An LSTM has three of these gates.

Forget gate decides what information we’re going to throw away from the cell state. 

$f_t = \sigma(W_{if}x_t + W_{hf}h_{t-1}+b_f)$

$\sigma$ squashes input values between 0 and 1, describing how much of each component should be let through. Zero means "let nothing through", while a value of one means "let everything through".

Input gate decides what new information we are going to store in the cell state. 

$i_t = \sigma(W_{ii}x_t + W_{hi}h_{t-1}+b_i)$

Next, a tanh layer creates a vector of new candidate values, $g_t$, that could be added to the state. tanh squashes the output values to be between −1 and 1. 

$g_t = tanh(W_{ig}x_t + W_{hg}h_{t-1}+b_g)$ (this equation equal to vanilla RNN if we remove gates)

The next step combines these two to create an update to the state. Pointwise multiplication operation (*) decides on the parts we output.

$c_t = f_t * c_{t-1} + i_t * g_t$

Finally, the output gate decides how much information goes to the output:

$o_t = \sigma(W_{io}x_t + W_{ho}h_{t-1}+b_o)$

$h_t = o_t * tanh(c_t)$
 
**Q: How does this help with the vanishing gradient problem ?**

## LSTMs vs. GRUs

Gated Recurrent Unit (GRU) combines the forget and input gates into a single "update gate" (z). So we have only two gates: update and reset. It also merges the cell state and hidden state, and makes some other changes. The resulting model is simpler than standard LSTM models. Candidate state $g_t$ is able to suppress $h_t$. The final state is a convex combination: of the $g_t$ and $h_{t-1}$ with coefficients of $(1 - z_t)$ and $z_t$ respectively.

$r_t = \sigma(W_{ir}x_t + W_{hr}h_{t-1}+b_r)$

$z_t = \sigma(W_{iz}x_t + W_{hz}h_{t-1}+b_z)$

$g_t = tanh(W_{ig}x_t + r_t * (W_{hg}h_{t-1}+b_g))$

$h_t = (1 - z_t)* g_t + z_t * h_{t-1}$


**Q. Implement an LSTM cell from scratch. Compare its performance to RNN and LSTM from the Pytorch toolkit.**



In [ ]:
class RNN(nn.Module):
    
    # variant is a flag which is either: "rnn", "lstm", "manual_lstm"
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout, pad_idx, variant):
            
        super().__init__()
        
        self.variant = variant
        
        self.embedding = nn.Embedding.from_pretrained(TEXT.vocab.vectors)

        # UNIDIRECTIONAL RNN layer: For LM modelling we do not see/have access to the right context
        
        if variant == "rnn":
            self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        elif variant == "lstm":
            self.rnn = nn.LSTM(embedding_dim, 
                               hidden_dim, 
                               batch_first=True)
        elif variant == "my_lstm":
            self.rnn = My_LSTM(embedding_dim, hidden_dim)
        else:
            raise ValueError("Expected `variant` to be one of 'rnn', 'lstm', or 'my_lstm'")
            
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)

       
    def forward(self, text, prev_hidden):
         
        # shape(text) = [B, T]
        
        # If vanilla RNN:
            # shape(prev_hidden) = [1, B, D] where 1 = num_layers*num_directions
        # If LSTM:
            # prev_hidden is a tuple of previous hidden states and cell states: (ALL_HIDDEN_STATES, ALL_CELL_STATES)
            # shape(ALL_HIDDEN_STATES)=shape(ALL_CELL_STATES) = [1, B, D] where 1 = num_layers*num_directions
            
        embedded = self.dropout(self.embedding(text))
        # shape(embedded) = [B, T, E]
        
        all_hidden, last_hidden = self.rnn(embedded, prev_hidden)        
        # shape(all_hidden) = [B, T, D]
        # shape(last_hidden) = [num layers, B, D]
        # For a LSTM:
            # last_hidden = (h_t, c_t)
            # shape(h_t = c_t) = [num_layers, B, D]
            
        if self.variant == "lstm" or self.variant == "my_lstm":
            last_hidden = last_hidden[0]
        
        # Take all hidden states to produce an output word per time step
        
        logits = self.fc(self.dropout(last_hidden))
        # shape(logits) = [B, O]
            
        return logits, last_hidden

class My_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

  
    
    def forward(self, x, prev_hidden):

  
        return hidden_states, (h_t, c_t)
 
my_lstm = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM,
            DROPOUT, 
            PAD_IDX,
            variant="my_lstm")

lstm = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM,
            DROPOUT, 
            PAD_IDX,
            variant="lstm")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(my_lstm.parameters())
train(my_lstm, train_iterator, valid_iterator, optimizer, criterion, is_lstm=True, force_stop=True)

optimizer = optim.Adam(lstm.parameters())
train(lstm, train_iterator, valid_iterator, optimizer, criterion, is_lstm=True, force_stop=True)

# Assignment 3: Predicting Humor in Edited News Headlines

Please read and follow the [homework requirements](https://docs.google.com/document/d/1zupXePA0SOlBJ39mfd5fnTghOAJx5AdbsPoo_5sa6co/edit?usp=sharing).

Develop a regression model (different from Assignment 2) to assess Humor in Edited News Headlines (in English). Use the same dataset as you used in the previous assignment. Details on the task are outlined here: https://competitions.codalab.org/competitions/20970. 

You can use the code from lab sessions or write your own code. Final results are to be submitted to the online competition platform (submit to Post-Evaluation-Task-1). Pay particular attention to the submission format. The evaluation of results is done automatically by the platform. The result will be displayed publicly on the leaderboard. You will be able to download the output log with the evaluation results that you will have to attach to your homework submission.

Submit the code, your user name (as displayed on the leaderboard), scoring output log from your competition account and a short report (150 words) answering the following questions:  

1. Briefly describe your model. How the model is different to the model from Assignment 2? Why did you pick up this new model?
2. What is the final set hyperparameters? How did you tune them?
3. Did you manage to improve your previous performance? Report the previous and the current performance. If yes or no, please explain why.